<a href="https://colab.research.google.com/github/teamgaon/recommendation_algorithms_know/blob/main/220110_sm_num_to_text_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Classification Knowcode using KoBERT Model

할일 : 텍스트로 변형 가능한 데이터 찾기

예 : 최종학력, 전공, 성별, 등

In [1]:
!pip install gluonnlp
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
!pip install transformers
!pip install sentencepiece
!pip install kobert-transformers
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-1b5_xgw9/kobert-tokenizer_e6a1c41b55b04db68d43d084763d0c91
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-1b5_xgw9/kobert-tokenizer_e6a1c41b55b04db68d43d084763d0c91
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-3ftxlr1i
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-3ftxlr1i


In [2]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from kobert_tokenizer import KoBERTTokenizer
from sklearn.model_selection import train_test_split
from google.colab import drive
import matplotlib.pyplot as plt
import warnings

In [3]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
warnings.filterwarnings(action='ignore') 
train1 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2017.csv', encoding='utf-8')
train2 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2018.csv', encoding='utf-8')
train3 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2019.csv', encoding='utf-8')
train4 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2020.csv', encoding='utf-8')

test1 = pd.read_csv('/content/gdrive/MyDrive/know/test/KNOW_2017_test.csv', encoding='utf-8')
test2 = pd.read_csv('/content/gdrive/MyDrive/know/test/KNOW_2018_test.csv', encoding='utf-8')
test3 = pd.read_csv('/content/gdrive/MyDrive/know/test/KNOW_2019_test.csv', encoding='utf-8')
test4 = pd.read_csv('/content/gdrive/MyDrive/know/test/KNOW_2020_test.csv', encoding='utf-8')

submission = pd.read_csv('/content/gdrive/MyDrive/know/sample_submission.csv', encoding='utf-8')

train2 = train2[train2['idx'] != 18048]
train3 = train3[train3['idx'] != 19871]

## 2017 텍스트 데이터 전처리

In [5]:
def bq1_to_text(num:int):
  if num == 1:
    return '산업유형은 농업, 임업, 어업.'
  elif num == 2:
    return '산업유형은 광업.'
  elif num == 3:
    return '산업유형은 제조업.'
  elif num == 4:
    return '산업유형은 전기, 가스, 증기 및 공기조절 공급업.'
  elif num == 5:
    return '산업유형은 수도·하수·폐기물 처리, 원료재생업.'
  elif num == 6:
    return '산업유형은 건설업.'
  elif num == 7:
    return '산업유형은 도매 및 소매업.'
  elif num == 8:
    return '산업유형은 운수 및 창고업.'
  elif num == 9:
    return '산업유형은 숙박 및 음식점업.'
  elif num == 10:
    return '산업유형은 정보통신업.'
  elif num == 11:
    return '산업유형은 금융 및 보험업.'
  elif num == 12:
    return '산업유형은 부동산업.'
  elif num == 13:
    return '산업유형은 전문, 과학 및 기술 서비스업.'
  elif num == 14:
    return '산업유형은 사업시설 관리, 사업지원 및 임대 서비스업.'
  elif num == 15:
    return '산업유형은 공공행정, 국방 및 사회보장 행정.'
  elif num == 16:
    return '산업유형은 교육 서비스업.'
  elif num == 17:
    return '산업유형은 보건업 및 사회복지 서비스업.'
  elif num == 18:
    return '산업유형은 예술, 스포츠 및 여가관련 서비스업.'
  elif num == 19:
    return '산업유형은 협회 및 단체, 수리 및 기타 개인 서비스업.'
  elif num == 20:
    return '산업유형은 가구내 고용활동 및 달리 분류되지 않은 자가소비 생산활동.'
  elif num == 21:
    return '산업유형은 국제 및 외국기관.'

train1['bq1'] = train1['bq1'].map(bq1_to_text)

In [6]:
def bq2_to_text(num:int):
  if num == 1:
    return '근무업체 유형은 민간회사 또는 개인사업체(외국인회사 포함).'
  elif num == 2:
    return '근무업체 유형은 정부기관 및 산하기관(공기업 포함).'
  elif num == 3:
    return '근무업체 유형은 (재단, 사단)법인단체.'
  elif num == 4:
    return '근무업체 유형은 특정회사나 사업체에 소속되어 있지 않음 (자영업 포함).'
  elif num == 5:
    return '근무업체 유형은 학교법인 혹은 학원.'
  elif num == 6:
    return '근무업체 유형은 기타.'

train1['bq2'] = train1['bq2'].map(bq2_to_text)

In [7]:
def bq3_to_text(num:int):
  if num == 1:
    return '근무업체 규모는 1명 ~ 4명.'
  elif num == 2:
    return '근무업체 규모는 5명 ~ 49명.'
  elif num == 3:
    return '근무업체 규모는 50명 ~ 99명.'
  elif num == 4:
    return '근무업체 규모는 100명 ~299명.'
  elif num == 5:
    return '근무업체 규모는 300명 ~ 999명.'
  elif num == 6:
    return '근무업체 규모는 1,000명 이상.'

train1['bq3'] = train1['bq3'].map(bq3_to_text)

In [8]:
def bq5_1_to_text(num:int):
  try :
    num = int(num)
    if num == 1:
      return '훈련기간은 1개월 미만.'
    elif num == 2:
      return '훈련기간은 1개월 이상~3개월 미만.'
    elif num == 3:
      return '훈련기간은 3개월 이상~6개월 미만.'
    elif num == 4:
      return '훈련기간은 6개월 이상~1년 미만.'
    elif num == 5:
      return '훈련기간은 1년 이상~2년 미만.'
    elif num == 6:
      return '훈련기간은 2년 이상.'
  except :
    return '훈련기간은 필요없다.'

train1['bq5_1'] = train1['bq5_1'].map(bq5_1_to_text)

In [9]:
def bq6_to_text(num:int):
  if num == 1:
    return '관련된 업무경험은 1개월 미만.'
  elif num == 2:
    return '관련된 업무경험은 1개월 이상~3개월 미만.'
  elif num == 3:
    return '관련된 업무경험은 3개월 이상~6개월 미만.'
  elif num == 4:
    return '관련된 업무경험은 6개월 이상~1년 미만.'
  elif num == 5:
    return '관련된 업무경험은 1년 이상~2년 미만.'
  elif num == 6:
    return '관련된 업무경험은 2년 이상.'
  elif num == 7:
    return '이전 업무경험 필요없음.'

train1['bq6'] = train1['bq6'].map(bq6_to_text)

In [10]:
def bq7_to_text(num:int):
  if num == 1:
    return '업무 수행 교육수준은 중학교 졸업 이하.'
  elif num == 2:
    return '업무 수행 교육수준은 고등학교 졸업.'
  elif num == 3:
    return '업무 수행 교육수준은 고등학교 졸업 후 직업교육(6개월 이상) 이수.'
  elif num == 4:
    return '업무 수행 교육수준은 전문대학교 졸업.'
  elif num == 5:
    return '업무 수행 교육수준은 대학교 졸업.'
  elif num == 6:
    return '업무 수행 교육수준은 대학원 석사 졸업.'
  elif num == 7:
    return '업무 수행 교육수준은 대학원 박사 졸업.'

train1['bq7'] = train1['bq7'].map(bq7_to_text)

In [11]:
def bq17_to_text(num:int):
  if num == 1:
    return '업무성과 극대화 연령대은 20대 이하.'
  elif num == 2:
    return '업무성과 극대화 연령대은 30대.'
  elif num == 3:
    return '업무성과 극대화 연령대은 40대.'
  elif num == 4:
    return '업무성과 극대화 연령대은 50대.'
  elif num == 5:
    return '업무성과 극대화 연령대은 60대 이상.'

train1['bq17'] = train1['bq17'].map(bq17_to_text)

In [12]:
def bq23_to_text(num:int):
  return '기술적 변화요인으로 인한 업무 대체 비율은 ' + str(num) + '%.'

train1['bq23'] = train1['bq23'].map(bq23_to_text)

In [13]:
def bq36_to_text(num:int):
  if num == 1:
    return '나의 성별은 남성이다.'
  elif num == 2:
    return '나의 성별은 여성이다.'

train1['bq36'] = train1['bq36'].map(bq36_to_text)

In [14]:
def bq37_to_text(num:int):
  return '나의 나이는 ' + str(num) + '살이다.'

train1['bq37'] = train1['bq37'].map(bq37_to_text)

In [15]:
def bq38_to_text(num:int):
  if num == 1:
    return '나의 최종학력은 중학교 졸업 이하이다.'
  elif num == 2:
    return '나의 최종학력은 고등학교 졸업이다.'
  elif num == 3:
    return '나의 최종학력은 2~3년제 대학교 졸업이다.'
  elif num == 4:
    return '나의 최종학력은 대학교 졸업이다.'
  elif num == 5:
    return '나의 최종학력은 대학원 석사 졸업이다.'
  elif num == 6:
    return '나의 최종학력은 대학원 박사 졸업이다.'

train1['bq38'] = train1['bq38'].map(bq38_to_text)

In [16]:
def bq40_to_text(num:int):
  try:
    num = int(num)
    if num == 1:
      return '나의 계약형태는 정규직이다.'
    elif num == 2:
      return '나의 계약형태는 비정규직이다.'
  except:
    return '나의 계약형태는 알 수 없다.'

train1['bq40'] = train1['bq40'].map(bq40_to_text)

In [17]:
def bq41_1_to_text(num:int):
  try:
    num = int(num)
    return '나의 근로소득은 ' + str(num) + '만원이다.'
  except:
    return '나의 근로소득은 알 수 없다.'

train1['bq41_1'] = train1['bq41_1'].map(bq41_1_to_text)

In [18]:
def bq41_2_to_text(num:int):
  try:
    num = int(num)
    return '나의 초임임금은 ' + str(num) + '만원이다.'
  except:
    return '나의 초임임금은 알 수 없다.'

train1['bq41_2'] = train1['bq41_2'].map(bq41_2_to_text)

In [19]:
def bq41_3_to_text(num:int):
  try:
    num = int(num)
    return '나의 순수입은 ' + str(num) + '만원이다.'
  except:
    return '나의 순수입은 알 수 없다.'

train1['bq41_3'] = train1['bq41_3'].map(bq41_3_to_text)

In [20]:
train1.head()

,idx,aq1_1,aq1_2,aq2_1,aq2_2,aq3_1,aq3_2,aq4_1,aq4_2,aq5_1,aq5_2,aq6_1,aq6_2,aq7_1,aq7_2,aq8_1,aq8_2,aq9_1,aq9_2,aq10_1,aq10_2,aq11_1,aq11_2,aq12_1,aq12_2,aq13_1,aq13_2,aq14_1,aq14_2,aq15_1,aq15_2,aq16_1,aq16_2,aq17_1,aq17_2,aq18_1,aq18_2,aq19_1,aq19_2,aq20_1,...,bq18_4,bq18_5,bq18_6,bq18_7,bq19,bq19_1,bq20,bq21,bq22,bq23,bq24_1,bq24_2,bq24_3,bq24_4,bq24_5,bq24_6,bq24_7,bq24_8,bq25,bq26,bq27,bq28,bq29,bq30,bq31,bq32,bq33,bq34,bq35,bq36,bq37,bq38,bq38_1,bq39_1,bq39_2,bq40,bq41_1,bq41_2,bq41_3,knowcode
0,0,3,3,3,3,3,3,4,4,3,4,3,3,2,2,2,2,2,2,3,3,3,3,4,4,3,3,3,2,3,3,5,6,5,6,4,5,2,2,5,...,3,2,3,3,2,생산설비의 자동화로,2,3,5,기술적 변화요인으로 인한 업무 대체 비율은 10.0%.,1,1,1,2,2,2,2,2,3,3,4,3,2,없다,없다,없다,건설현장 노무직,없다,4,나의 성별은 남성이다.,나의 나이는 52살이다.,나의 최종학력은 고등학교 졸업이다.,실업,1,1,나의 계약형태는 정규직이다.,나의 근로소득은 4000만원이다.,나의 초임임금은 알 수 없다.,나의 순수입은 2200만원이다.,825101
1,1,4,5,4,5,3,4,3,4,3,4,1,,1,,1,,1,,2,3,3,4,3,4,4,5,2,2,3,4,3,4,3,4,3,4,1,,2,...,4,2,2,4,2,건설 수주가 없어서,3,3,6,기술적 변화요인으로 인한 업무 대체 비율은 50.0%.,2,2,2,2,2,2,2,2,4,3,4,3,2,없다,"매타기, 드라이버, 가위, CAD",,,,2,나의 성별은 남성이다.,나의 나이는 38살이다.,나의 최종학력은 대학교 졸업이다.,건축공학,1,1,나의 계약형태는 정규직이다.,나의 근로소득은 알 수 없다.,나의 초임임금은 알 수 없다.,나의 순수입은 2400만원이다.,140204
2,2,3,4,3,4,3,4,5,6,4,5,1,,1,,3,4,3,4,3,4,1,,3,4,3,4,1,,3,4,3,4,3,4,3,4,3,4,3,...,4,2,2,4,2,신축 건설경기가 좋지 않아서,3,2,5,기술적 변화요인으로 인한 업무 대체 비율은 40.0%.,2,2,2,2,2,2,2,2,4,4,4,2,2,없다,"건축설계표, 오토캐드, 파이프, 보일러",,,,2,나의 성별은 남성이다.,나의 나이는 50살이다.,나의 최종학력은 대학교 졸업이다.,건축공학,1,1,나의 계약형태는 정규직이다.,나의 근로소득은 4000만원이다.,나의 초임임금은 알 수 없다.,나의 순수입은 2400만원이다.,140204
3,3,3,3,3,3,3,5,4,5,4,6,3,5,3,4,4,5,5,5,4,5,3,4,4,5,3,5,3,4,4,4,3,4,2,2,4,4,4,2,4,...,3,2,2,3,1,"업무량 감소, 자동화로 인해서",4,2,6,기술적 변화요인으로 인한 업무 대체 비율은 30.0%.,2,1,1,2,1,1,1,1,3,4,4,4,4,지적기사,"토탈스테이션, SZP",무,지적공무원,,4,나의 성별은 남성이다.,나의 나이는 42살이다.,나의 최종학력은 대학교 졸업이다.,환경학과,1,1,나의 계약형태는 정규직이다.,나의 근로소득은 7000만원이다.,나의 초임임금은 알 수 없다.,나의 순수입은 3500만원이다.,140601
4,4,4,5,3,4,3,4,4,5,3,4,1,,1,,1,,1,,3,4,2,2,3,4,3,4,1,,3,4,3,4,3,4,3,4,1,,2,...,4,2,2,4,2,부동산 거품 빠지면서 건축 경기도 덩달아 불황,3,2,3,기술적 변화요인으로 인한 업무 대체 비율은 40.0%.,2,2,2,2,2,2,2,2,4,3,4,3,3,없다,"캐드, 엑셀, 건축도면",,,,2,나의 성별은 남성이다.,나의 나이는 51살이다.,나의 최종학력은 대학교 졸업이다.,건축공학,1,1,나의 계약형태는 정규직이다.,나의 근로소득은 4000만원이다.,나의 초임임금은 알 수 없다.,나의 순수입은 2500만원이다.,140204


## 2018 텍스트 데이터 전처리

In [21]:
def cq48_to_text(num:int):
  if num == 1:
    return '나는 재택근무가 불가능하다.'
  elif num == 2:
    return '나는 재택근무가 가능하다.'

train2['cq48'] = train2['cq48'].map(cq48_to_text)

In [22]:
def bq1_to_text(num:int):
  if num == 1:
    return '산업유형은 농업, 임업, 어업.'
  elif num == 2:
    return '산업유형은 광업.'
  elif num == 3:
    return '산업유형은 제조업.'
  elif num == 4:
    return '산업유형은 전기, 가스, 증기 및 공기조절 공급업.'
  elif num == 5:
    return '산업유형은 수도·하수·폐기물 처리, 원료재생업.'
  elif num == 6:
    return '산업유형은 건설업.'
  elif num == 7:
    return '산업유형은 도매 및 소매업.'
  elif num == 8:
    return '산업유형은 운수 및 창고업.'
  elif num == 9:
    return '산업유형은 숙박 및 음식점업.'
  elif num == 10:
    return '산업유형은 정보통신업.'
  elif num == 11:
    return '산업유형은 금융 및 보험업.'
  elif num == 12:
    return '산업유형은 부동산업.'
  elif num == 13:
    return '산업유형은 전문, 과학 및 기술 서비스업.'
  elif num == 14:
    return '산업유형은 사업시설 관리, 사업지원 및 임대 서비스업.'
  elif num == 15:
    return '산업유형은 공공행정, 국방 및 사회보장 행정.'
  elif num == 16:
    return '산업유형은 교육 서비스업.'
  elif num == 17:
    return '산업유형은 보건업 및 사회복지 서비스업.'
  elif num == 18:
    return '산업유형은 예술, 스포츠 및 여가관련 서비스업.'
  elif num == 19:
    return '산업유형은 협회 및 단체, 수리 및 기타 개인 서비스업.'
  elif num == 20:
    return '산업유형은 가구내 고용활동 및 달리 분류되지 않은 자가소비 생산활동.'
  elif num == 21:
    return '산업유형은 국제 및 외국기관.'

train2['bq1'] = train2['bq1'].map(bq1_to_text)

In [23]:
def bq2_to_text(num:int):
  if num == 1:
    return '근무업체 유형은 민간회사 또는 개인사업체(외국인회사 포함).'
  elif num == 2:
    return '근무업체 유형은 정부기관 및 산하기관(공기업 포함).'
  elif num == 3:
    return '근무업체 유형은 (재단, 사단)법인단체.'
  elif num == 4:
    return '근무업체 유형은 특정회사나 사업체에 소속되어 있지 않음 (자영업 포함).'
  elif num == 5:
    return '근무업체 유형은 학교법인 혹은 학원.'
  elif num == 6:
    return '근무업체 유형은 기타.'

train2['bq2'] = train2['bq2'].map(bq2_to_text)

In [24]:
def bq3_to_text(num:int):
  if num == 1:
    return '근무업체 규모는 1명 ~ 4명.'
  elif num == 2:
    return '근무업체 규모는 5명 ~ 49명.'
  elif num == 3:
    return '근무업체 규모는 50명 ~ 99명.'
  elif num == 4:
    return '근무업체 규모는 100명 ~299명.'
  elif num == 5:
    return '근무업체 규모는 300명 ~ 999명.'
  elif num == 6:
    return '근무업체 규모는 1,000명 이상.'

train2['bq3'] = train2['bq3'].map(bq3_to_text)

In [25]:
def bq5_1_to_text(num:int):
  try :
    num = int(num)
    if num == 1:
      return '훈련기간은 1개월 미만.'
    elif num == 2:
      return '훈련기간은 1개월 이상~3개월 미만.'
    elif num == 3:
      return '훈련기간은 3개월 이상~6개월 미만.'
    elif num == 4:
      return '훈련기간은 6개월 이상~1년 미만.'
    elif num == 5:
      return '훈련기간은 1년 이상~2년 미만.'
    elif num == 6:
      return '훈련기간은 2년 이상.'
  except :
    return '훈련기간은 필요없다.'

train2['bq5_1'] = train2['bq5_1'].map(bq5_1_to_text)

In [26]:
def bq13_to_text(num:int):
  if num == 1:
    return '업무내용 규정성은 사업주/상사에게 있다.'
  elif num == 2:
    return '업무내용 규정성은 고객에게 있다.'
  elif num == 3:
    return '업무내용 규정성은 본인재량이다.'

train2['bq13'] = train2['bq13'].map(bq13_to_text)

In [27]:
def bq14_to_text(num:int):
  if num == 1:
    return '나는 업무관련 수칙을 적용 받는다.'
  elif num == 2:
    return '나는 업무관련 수칙을 적용 받지 않는다.'

train2['bq14'] = train2['bq14'].map(bq14_to_text)

In [28]:
def bq15_to_text(num:int):
  if num == 1:
    return '지휘감독은 사업주/상사에게 받는다.'
  elif num == 2:
    return '지휘감독은 지점/대리점의 점장이나 소장에게 받는다.'
  elif num == 3:
    return '지휘감독은 고객에게 받는다.'
  elif num == 4:
    return '지휘감독을 받지 않는다.'

train2['bq15'] = train2['bq15'].map(bq15_to_text)

In [29]:
def bq16_to_text(num:int):
  if num == 1:
    return '나는 업무 거부가 가능하다.'
  elif num == 2:
    return '나는 업무 거부가 불가능하다.'

train2['bq16'] = train2['bq16'].map(bq16_to_text)

In [30]:
def bq17_to_text(num:int):
  if num == 1:
    return '근무시간과 장소는 주로 사업주/상사에 의해 정해진다.'
  elif num == 2:
    return '근무시간과 장소는 주로 고객에 의해 정해진다.'
  elif num == 3:
    return '근무시간과 장소는 주로 본인 재량에 의해 정해진다.'

train2['bq17'] = train2['bq17'].map(bq17_to_text)

In [31]:
def bq18_to_text(num:int):
  if num == 1:
    return '작업도구는 주로 사업주/상사 소유다.'
  elif num == 2:
    return '작업도구는 사업주와 본인이 반반 소유한다.'
  elif num == 3:
    return '작업도구는 주로 본인이 소유한다.'

train2['bq18'] = train2['bq18'].map(bq18_to_text)

In [32]:
def bq21_to_text(num:int):
  return '나의 보수 중 고정급은 ' + str(num) + '%이다.'

train2['bq21'] = train2['bq21'].map(bq21_to_text)

In [33]:
def bq27_to_text(num:int):
  if num == 1:
    return '나는 플랫폼 중개로 일거리를 얻는다.'
  elif num == 2:
    return '나는 플랫폼 중개로 일거리를 얻지 않는다.'

train2['bq27'] = train2['bq27'].map(bq27_to_text)

In [34]:
train2.head()

,idx,cq1,cq2,cq3,cq4,cq5,cq6,cq7,cq8,cq9,cq10,cq11,cq12,cq13,cq14,cq15,cq16,cq17,cq18,cq19,cq20,cq21,cq22,cq23,cq24,cq25,cq26,cq27,cq28,cq29,cq30,cq31,cq32,cq33,cq34,cq35,cq36,cq37,cq38,cq39,...,bq233,bq234,bq235,bq241,bq242,bq243,bq244,bq245,bq25,bq25_1,bq26_1,bq26_1a,bq26_2,bq26_2a,bq26_3,bq26_3a,bq26_4,bq26_4a,bq27,bq28,bq28_1,bq29,bq30,bq31,bq32,bq33,bq34,bq35,bq36,bq37,bq37_1,bq38,bq38_1,bq38_2,bq39,bq40,bq41_1,bq41_2,bq41_3,knowcode
0,9486,5,3,4,2,2,5,4,5,4,4,4,3,3,4,1,5,3,4.0,2.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,2.0,4.0,2.0,1.0,1.0,2.0,4.0,3.0,4.0,4.0,3.0,4.0,...,0.0,0,,1,,,,,2,,1,2,1,2,1,1,1,1,나는 플랫폼 중개로 일거리를 얻지 않는다.,4,정교한 직업 요구의 증가,치기공사,치카공,,교정전문가,,3,1,27,3,치기공과,1,1,,1,42,3800,3400,,306301
1,9487,4,3,4,4,4,4,4,4,4,3,3,3,3,4,3,2,2,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4.0,2.0,2.0,1.0,1.0,3.0,1.0,3.0,3.0,3.0,3.0,...,0.0,0,,,,3,,,1,2,3,,3,,3,,3,,나는 플랫폼 중개로 일거리를 얻는다.,2,게임 산업의 축소로 인해,없다,"서브라임텍스트, 유니티, 리눅스",정보보호컨설턴트,일반 프로그래머,없다,3,1,28,2,이과,1,2,,1,40,2300,1800,,133204
2,9488,4,1,5,3,3,4,2,4,1,2,1,2,2,2,2,1,1,2.0,2.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,2.0,3.0,2.0,4.0,1.0,4.0,3.0,3.0,1.0,4.0,4.0,5.0,4.0,...,0.0,0,5,,,,,5,3,,2,,2,,2,,2,,나는 플랫폼 중개로 일거리를 얻지 않는다.,2,대형화 되는 추세,사장님,망치 실리콘 사다리 유리칼,회사원,,,4,1,59,2,기계과,2,,6,,30.0,,,3000,702502
3,9489,4,3,4,4,4,3,3,2,3,3,2,3,3,4,4,3,3,2.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,4.0,3.0,4.0,4.0,2.0,2.0,2.0,2.0,4.0,4.0,3.0,4.0,3.0,3.0,...,0.0,0,,1,,,,,1,2,1,2,1,2,1,2,1,1,나는 플랫폼 중개로 일거리를 얻지 않는다.,3,"과학이 발달할수록 실험,검사,분석 보조가 필요하므로",,전자현미경 컴퓨터,,,,2,1,38,4,물리 과학,1,1,,1,40,3350,2750,,121201
4,9490,3,2,3,1,1,4,3,4,3,2,3,1,1,5,2,3,4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,4.0,2.0,2.0,3.0,4.0,4.0,3.0,1.0,1.0,1.0,1.0,...,0.0,0,,1,,,,,1,1,1,2,1,2,1,2,1,1,나는 플랫폼 중개로 일거리를 얻지 않는다.,4,고령화로인하여 틀니 및 인플란트등 의료보험 보장범위가 확대되어서,,"니켈, 크롬, 코발트",판매업,,,4,2,27,4,치기공학,1,1,,1,40,2500,2300,,306301


## 2019 텍스트 데이터 전처리

In [35]:
def bq1_to_text(num:int):
  if num == 1:
    return '산업유형은 농업, 임업, 어업.'
  elif num == 2:
    return '산업유형은 광업.'
  elif num == 3:
    return '산업유형은 제조업.'
  elif num == 4:
    return '산업유형은 전기, 가스, 증기 및 공기조절 공급업.'
  elif num == 5:
    return '산업유형은 수도·하수·폐기물 처리, 원료재생업.'
  elif num == 6:
    return '산업유형은 건설업.'
  elif num == 7:
    return '산업유형은 도매 및 소매업.'
  elif num == 8:
    return '산업유형은 운수 및 창고업.'
  elif num == 9:
    return '산업유형은 숙박 및 음식점업.'
  elif num == 10:
    return '산업유형은 정보통신업.'
  elif num == 11:
    return '산업유형은 금융 및 보험업.'
  elif num == 12:
    return '산업유형은 부동산업.'
  elif num == 13:
    return '산업유형은 전문, 과학 및 기술 서비스업.'
  elif num == 14:
    return '산업유형은 사업시설 관리, 사업지원 및 임대 서비스업.'
  elif num == 15:
    return '산업유형은 공공행정, 국방 및 사회보장 행정.'
  elif num == 16:
    return '산업유형은 교육 서비스업.'
  elif num == 17:
    return '산업유형은 보건업 및 사회복지 서비스업.'
  elif num == 18:
    return '산업유형은 예술, 스포츠 및 여가관련 서비스업.'
  elif num == 19:
    return '산업유형은 협회 및 단체, 수리 및 기타 개인 서비스업.'
  elif num == 20:
    return '산업유형은 가구내 고용활동 및 달리 분류되지 않은 자가소비 생산활동.'
  elif num == 21:
    return '산업유형은 국제 및 외국기관.'

train3['bq1'] = train3['bq1'].map(bq1_to_text)

In [36]:
def bq2_to_text(num:int):
  if num == 1:
    return '근무업체 유형은 민간회사 또는 개인사업체(외국인회사 포함).'
  elif num == 2:
    return '근무업체 유형은 정부기관 및 산하기관(공기업 포함).'
  elif num == 3:
    return '근무업체 유형은 (재단, 사단)법인단체.'
  elif num == 4:
    return '근무업체 유형은 특정회사나 사업체에 소속되어 있지 않음 (자영업 포함).'
  elif num == 5:
    return '근무업체 유형은 학교법인 혹은 학원.'
  elif num == 6:
    return '근무업체 유형은 기타.'

train3['bq2'] = train3['bq2'].map(bq2_to_text)

In [37]:
def bq3_to_text(num:int):
  if num == 1:
    return '근무업체 규모는 1명 ~ 4명.'
  elif num == 2:
    return '근무업체 규모는 5명 ~ 49명.'
  elif num == 3:
    return '근무업체 규모는 50명 ~ 99명.'
  elif num == 4:
    return '근무업체 규모는 100명 ~299명.'
  elif num == 5:
    return '근무업체 규모는 300명 ~ 999명.'
  elif num == 6:
    return '근무업체 규모는 1,000명 이상.'

train3['bq3'] = train3['bq3'].map(bq3_to_text)


In [38]:
def bq5_1_to_text(num:int):
  try :
    num = int(num)
    if num == 1:
      return '훈련기간은 1개월 미만.'
    elif num == 2:
      return '훈련기간은 1개월 이상~3개월 미만.'
    elif num == 3:
      return '훈련기간은 3개월 이상~6개월 미만.'
    elif num == 4:
      return '훈련기간은 6개월 이상~1년 미만.'
    elif num == 5:
      return '훈련기간은 1년 이상~2년 미만.'
    elif num == 6:
      return '훈련기간은 2년 이상.'
  except :
    return '훈련기간은 필요없다.'

train3['bq5_1'] = train3['bq5_1'].map(bq5_1_to_text)

In [39]:
train3.head()

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,sq10,sq11,sq12,sq13,sq14,sq15,sq16,kq1_1,kq1_2,kq2_1,kq2_2,kq3_1,kq3_2,kq4_1,kq4_2,kq5_1,kq5_2,kq6_1,kq6_2,kq7_1,kq7_2,kq8_1,kq8_2,kq9_1,kq9_2,kq10_1,kq10_2,kq11_1,kq11_2,kq12_1,...,bq14_5,bq15,bq16_1,bq16_2,bq16_3,bq16_4,bq16_5,bq17,bq18_1,bq18_2,bq18_3,bq18_4,bq18_5,bq18_6,bq18_7,bq18_8,bq18_9,bq18_10,bq19,bq20,bq20_1,bq21_1,bq21_2,bq21_3,bq22,bq23,bq24,bq25,bq26,bq27,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3,knowcode
0,18569,4,4,4,3,4,4,4,4,4,4,4,5,4,4,4,4,3,3,2,2,1,0,2,2,2,2,3,4,4,5,1,0,4,5,4,5,4,5,4,...,5.0,4.0,4.0,4.0,4.0,3.0,4.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,사람은 줄고 일은 많으니까 업무가 많다,3.0,4,수송수요가 늘어날다,4.0,4.0,3,,,,1,35.0,4,기계공학과,1.0,1,,1,40,3500,3000,,812301
1,18570,4,3,4,4,4,4,3,4,5,4,3,5,4,4,1,2,4,6,3,4,4,5,5,5,3,4,5,6,1,0,1,0,1,0,1,0,1,0,1,...,3.0,3.0,4.0,2.0,3.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,,1.0,3,운송시장이 큰 변화 없을듯해서,2.0,3.0,3,"컨테이너 대여사업,랜탈사업",,,1,63.0,3,경영학,2.0,,5,,40.0,,,5000,15201
2,18571,2,3,2,2,2,2,2,2,3,3,3,2,2,2,2,2,2,2,1,0,3,4,2,3,2,2,1,0,1,0,3,3,1,0,1,0,1,0,1,...,4.0,1.0,3.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,,1.0,3,,4.0,4.0,4,숙박업,,,2,62.0,2,,2.0,,6,,30,,,3000,901101
3,18572,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,7,5,6,5,6,5,6,4,5,4,5,4,5,5,6,3,3,3,3,2,...,4.0,3.0,4.0,3.0,3.0,3.0,3.0,4.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,,2.0,4,편리하게 일처리을 하기위해서 문서대행 업무를 맡길거 같아서,2.0,3.0,3,,,,2,33.0,4,행정학,1.0,1,,1,40.0,3600,2400,,29903
4,18573,1,4,4,1,1,2,4,3,4,4,4,5,4,3,1,1,2,2,2,2,2,2,2,2,4,5,1,0,1,0,1,0,1,0,1,0,1,0,1,...,3.0,3.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,,2.0,2,일거리가 줄어들고 있다,3.0,2.0,2,용역회사(청소원),,,1,67.0,1,,2.0,,6,,45,,,1500,561401


## 2020년 텍스트 데이터 전처리

In [40]:
train4['bq1'] = train4['bq1'].map(bq1_to_text)

In [41]:
train4['bq2'] = train4['bq2'].map(bq2_to_text)

In [42]:
train4['bq3'] = train4['bq3'].map(bq3_to_text)

In [43]:
train4['bq5_1'] = train4['bq5_1'].map(bq5_1_to_text)

In [44]:
def bq6_to_text(num:int):
  if num == 1:
    return '관련된 업무 경험은 1개월 미만.'
  elif num == 2:
    return '관련된 업무 경험은 1개월 이상~3개월 미만.'
  elif num == 3:
    return '관련된 업무 경험은 3개월 이상~6개월 미만.'
  elif num == 4:
    return '관련된 업무 경험은 6개월 이상~1년 미만.'
  elif num == 5:
    return '관련된 업무 경험은 1년 이상~2년 미만.'
  elif num == 6:
    return '관련된 업무 경험은 2년 이상.'
  elif num == 7:
    return '관련된 업무 경험은 필요없음.'

train4['bq6'] = train4['bq6'].map(bq6_to_text)

In [45]:
def bq7_to_text(num:int):
  if num == 1:
    return '업무 수행 교육수준은 중학교 졸업 이하.'
  elif num == 2:
    return '업무 수행 교육수준은 고등학교 졸업.'
  elif num == 3:
    return '업무 수행 교육수준은 고등학교 졸업 후 직업교육(6개월 이상) 이수.'
  elif num == 4:
    return '업무 수행 교육수준은 전문대학교 졸업.'
  elif num == 5:
    return '업무 수행 교육수준은 대학교 졸업.'
  elif num == 6:
    return '업무 수행 교육수준은 대학원 석사 졸업.'
  elif num == 7:
    return '업무 수행 교육수준은 대학원 박사 졸업.'

train4['bq7'] = train4['bq7'].map(bq7_to_text)

In [46]:
def bq24_to_text(num:int):
  if num == 1:
    return '나의 성별은 남성이다.'
  elif num == 2:
    return '나의 성별은 여성이다.'

train4['bq24'] = train4['bq24'].map(bq24_to_text)

In [47]:
def bq25_to_text(num:int):
  return '나의 나이는 ' + str(num) + '살이다.'

train4['bq25'] = train4['bq25'].map(bq25_to_text)

In [48]:
def bq26_to_text(num:int):
  if num == 1:
    return '나의 최종학력은 중학교 졸업 이하이다.'
  elif num == 2:
    return '나의 최종학력은 고등학교 졸업이다.'
  elif num == 3:
    return '나의 최종학력은 2~3년제 대학교 졸업이다.'
  elif num == 4:
    return '나의 최종학력은 대학교 졸업이다.'
  elif num == 5:
    return '나의 최종학력은 대학원 석사 졸업이다.'
  elif num == 6:
    return '나의 최종학력은 대학원 박사 졸업이다.'

train4['bq26'] = train4['bq26'].map(bq26_to_text)

In [49]:
def bq28_to_text(num:int):
  try:
    num = int(num)
    if num == 1:
      return '나의 계약형태는 정규직이다.'
    elif num == 2:
      return '나의 계약형태는 비정규직이다.'
  except:
    return '나의 계약형태는 알 수 없다.'

train4['bq28'] = train4['bq28'].map(bq28_to_text)

In [50]:
def bq29_to_text(num:int):
  return '주당 근로시간은 ' + str(num) + '시간이다.'

train4['bq29'] = train4['bq29'].map(bq29_to_text)

In [51]:
def bq30_1_to_text(num:int):
  try:
    num = int(num)
    return '나의 근로소득은 ' + str(num) + '만원이다.'
  except:
    return '나의 근로소득은 알 수 없다.'

train4['bq30_1'] = train4['bq30_1'].map(bq30_1_to_text)

In [52]:
def bq30_2_to_text(num:int):
  try:
    num = int(num)
    return '나의 초임임금은 ' + str(num) + '만원이다.'
  except:
    return '나의 초임임금은 알 수 없다.'

train4['bq30_2'] = train4['bq30_2'].map(bq30_2_to_text)

In [53]:
def bq30_3_to_text(num:int):
  try:
    num = int(num)
    return '나의 순수입은 ' + str(num) + '만원이다.'
  except:
    return '나의 순수입은 알 수 없다.'

train4['bq30_3'] = train4['bq30_3'].map(bq30_3_to_text)

In [54]:
train4.head()

,idx,saq1_1,saq1_2,saq2_1,saq2_2,saq3_1,saq3_2,saq4_1,saq4_2,saq5_1,saq5_2,saq6_1,saq6_2,saq7_1,saq7_2,saq8_1,saq8_2,saq9_1,saq9_2,saq10_1,saq10_2,saq11_1,saq11_2,saq12_1,saq12_2,saq13_1,saq13_2,saq14_1,saq14_2,saq15_1,saq15_2,saq16_1,saq16_2,saq17_1,saq17_2,saq18_1,saq18_2,saq19_1,saq19_2,saq20_1,...,bq16_10,bq17,bq18_1,bq18_2,bq18_3,bq18_4,bq18_5,bq18_6,bq18_7,bq18_8,bq18_9,bq18_10,bq19,bq20,bq20_1,bq21_1,bq21_2,bq21_3,bq21_4,bq22_1,bq22_2,bq22_3,bq22_4,bq22_5,bq22_6,bq23_1,bq23_2,bq23_3,bq24,bq25,bq26,bq26_1,bq27_1,bq27_2,bq28,bq29,bq30_1,bq30_2,bq30_3,knowcode
0,27127,4,4,4,4,4,5,4,5,3,3,4,5,1,,4,5,3,4,1,,3,4,3,4,4,5,3,4,4,5,3,4,3,4,1,,1,,4,...,4,4,2,2,2,2,2,2,2,2,2,,1,3,일정하게 인원이 정해져 있으므로 일자리 변화는 없을 것 같아서,3,2,3,2,5,7,4,5,2,3,3,4,3,나의 성별은 여성이다.,나의 나이는 42살이다.,나의 최종학력은 대학교 졸업이다.,간호학,1,1,나의 계약형태는 정규직이다.,주당 근로시간은 40시간이다.,나의 근로소득은 4000만원이다.,나의 초임임금은 2600만원이다.,나의 순수입은 알 수 없다.,304003
1,27128,5,6,5,6,4,5,4,5,4,5,4,5,5,6,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,4,5,3,4,4,...,4,3,2,2,2,2,2,2,2,2,2,,1,2,경기 침체,3,3,3,2,3,4,3,5,6,6,3,6,5,나의 성별은 남성이다.,나의 나이는 45살이다.,나의 최종학력은 대학교 졸업이다.,기계공학,1,1,나의 계약형태는 정규직이다.,주당 근로시간은 40시간이다.,나의 근로소득은 4000만원이다.,나의 초임임금은 3000만원이다.,나의 순수입은 알 수 없다.,221401
2,27129,3,4,3,4,3,4,3,5,3,5,3,5,3,5,4,5,4,5,4,5,3,4,3,4,3,4,4,5,4,4,4,5,3,4,4,5,4,5,3,...,4,3,2,2,1,1,2,2,2,2,2,"새로운 장비, 기술 도입됨",2,2,전체적으로 경기 악화 때문,3,2,3,2,5,7,5,5,5,5,5,5,5,나의 성별은 남성이다.,나의 나이는 38살이다.,나의 최종학력은 대학교 졸업이다.,기계,1,1,나의 계약형태는 정규직이다.,주당 근로시간은 48시간이다.,나의 근로소득은 4300만원이다.,나의 초임임금은 3000만원이다.,나의 순수입은 알 수 없다.,815001
3,27130,4,5,2,3,3,3,4,5,3,4,3,5,4,4,3,3,3,2,1,,2,3,3,4,3,5,3,2,3,3,4,4,3,4,1,,5,6,3,...,2,2,2,2,2,2,2,2,2,2,2,,1,3,사람의 상태를 소통하며 확인해서 그에 맞는 각가의 처방을 내려줘야 하는 일이라 대체불가,3,3,3,2,6,4,7,3,6,7,7,6,4,나의 성별은 여성이다.,나의 나이는 25살이다.,나의 최종학력은 대학교 졸업이다.,보건학,1,1,나의 계약형태는 정규직이다.,주당 근로시간은 40시간이다.,나의 근로소득은 3500만원이다.,나의 초임임금은 3300만원이다.,나의 순수입은 알 수 없다.,304003
4,27131,5,6,4,5,3,5,4,4,4,6,4,5,4,5,5,6,4,5,4,5,4,5,4,5,5,6,4,5,1,,1,,4,5,4,5,4,5,5,...,5,4,2,2,2,2,2,2,2,2,2,,2,2,건설경기가 아직은 침체에서 일어나기 힘들 것 같다,2,2,3,2,8,8,7,5,3,6,6,2,1,나의 성별은 남성이다.,나의 나이는 49살이다.,나의 최종학력은 대학교 졸업이다.,건축설비공학,1,1,나의 계약형태는 정규직이다.,주당 근로시간은 40시간이다.,나의 근로소득은 5700만원이다.,나의 초임임금은 2700만원이다.,나의 순수입은 알 수 없다.,140201


In [55]:
#bq4_1a, bq4_1b, bq4_1c, bq5_2, bq19_1, bq30, bq31, bq32, bq33, bq34, bq38_1
#bq1, bq2, bq3, bq5_1, bq6, bq7, bq17, bq23, bq36, bq37, bq38, bq40, bq41_1, bq41_2, bq41_3

cols = ['bq1', 'bq2', 'bq3', 'bq5_1', 'bq6', 'bq7', 'bq17', 'bq23', 'bq36', 'bq37', 'bq38', 'bq40', 'bq41_1', 'bq41_2', 'bq41_3',
        'bq4_1a', 'bq4_1b', 'bq4_1c', 'bq5_2', 'bq19_1', 'bq30', 'bq31', 'bq32', 'bq33', 'bq34', 'bq38_1']

def null_to_no(text : str):
  if text == ' ':
    return '없다'
  else:
    return text

for col in cols:
  train1[col] = train1[col].map(null_to_no)

train1['bq4_1a'] = '요구자격증은 ' + train1['bq4_1a'].astype(str) + '이다.'
train1['bq4_1b'] = '요구자격증은 ' + train1['bq4_1b'].astype(str) + '이다.'
train1['bq4_1c'] = '요구자격증은 ' + train1['bq4_1c'].astype(str) + '이다.'
train1['bq5_2'] = '요구훈련종류는 ' + train1['bq5_2'].astype(str) + '이다.'
train1['bq19_1'] = '향후 일자리 변화 이유는 ' + train1['bq19_1'].astype(str) + '이다.'
train1['bq30'] = '유사직업명은 ' + train1['bq30'].astype(str) + '이다.'
train1['bq31'] = '사용도구 및 프로그램은 ' + train1['bq31'].astype(str) + '이다.'
train1['bq32'] = '이전직업은 ' + train1['bq32'].astype(str) + '이다.'
train1['bq33'] = '전직 가능 직업은 ' + train1['bq33'].astype(str) + '이다.'
train1['bq34'] = '신직업 발굴은 ' + train1['bq34'].astype(str) + '이다.'
train1['bq38_1'] = '나의 전공은 ' + train1['bq38_1'].astype(str) + '이다.'

temp = ''
for col in cols:
  temp += train1[col]
  temp += ' '

train1['text'] = temp

In [56]:
# bq4_1a, bq4_1b, bq4_1c, bq5_2, bq28_1,	bq29,	bq30,	bq31,	bq32, bq33, bq37_1
# cq48, bq1, bq2, bq3, bq5_1, bq13, bq14, bq15, bq16, bq17, bq18, bq21, bq27

cols = ['cq48', 'bq1', 'bq2', 'bq3', 'bq5_1', 'bq13', 'bq14', 'bq15', 'bq16', 'bq17', 'bq18', 'bq21', 'bq27',
        'bq4_1a', 'bq4_1b', 'bq4_1c', 'bq5_2', 'bq28_1',	'bq29',	'bq30',	'bq31',	'bq32', 'bq33', 'bq37_1']

def null_to_no(text : str):
  if text == ' ':
    return '없다'
  else:
    return text

for col in cols:
  train2[col] = train2[col].map(null_to_no)

train2['bq4_1a'] = '요구자격증은 ' + train2['bq4_1a'].astype(str) + '이다.'
train2['bq4_1b'] = '요구자격증은 ' + train2['bq4_1b'].astype(str) + '이다.'
train2['bq4_1c'] = '요구자격증은 ' + train2['bq4_1c'].astype(str) + '이다.'
train2['bq5_2'] = '요구훈련종류는 ' + train2['bq5_2'].astype(str) + '이다.'
train2['bq28_1'] = '향후 일자리 변화 이유는 ' + train2['bq28_1'].astype(str) + '이다.'
train2['bq29'] = '유사직업명은 ' + train2['bq29'].astype(str) + '이다.'
train2['bq30'] = '사용도구 및 프로그램은 ' + train2['bq30'].astype(str) + '이다.'
train2['bq31'] = '이전직업은 ' + train2['bq31'].astype(str) + '이다.'
train2['bq32'] = '전직 가능 직업은 ' + train2['bq32'].astype(str) + '이다.'
train2['bq33'] = '신직업 발굴은 ' + train2['bq33'].astype(str) + '이다.'
train2['bq37_1'] = '나의 전공은 ' + train2['bq37_1'].astype(str) + '이다.'

temp = ''
for col in cols:
  temp += train2[col]
  temp += ' '

train2['text'] = temp

In [57]:
# bq4_1a, bq4_1b, bq4_1c, bq5_2, bq18_10,	bq20_1, bq22,	bq23,	bq24, bq27_1
# bq1, bq2, bq3, bq5_1

cols = ['bq1', 'bq2', 'bq3', 'bq5_1',
        'bq4_1a', 'bq4_1b', 'bq4_1c', 'bq5_2', 'bq18_10',	'bq20_1', 'bq22',	'bq23',	'bq24', 'bq27_1']

def null_to_no(text : str):
  if text == ' ':
    return '없다'
  else:
    return text

for col in cols:
  train3[col] = train3[col].map(null_to_no)

train3['bq4_1a'] = '요구자격증은 ' + train3['bq4_1a'].astype(str) + '이다.'
train3['bq4_1b'] = '요구자격증은 ' + train3['bq4_1b'].astype(str) + '이다.'
train3['bq4_1c'] = '요구자격증은 ' + train3['bq4_1c'].astype(str) + '이다.'
train3['bq5_2'] = '요구훈련종류는 ' + train3['bq5_2'].astype(str) + '이다.'
train3['bq18_10'] = '향후 일자리 변화 관련 예시는 ' + train3['bq18_10'].astype(str) + '이다.'
train3['bq20_1'] = '향후 일자리 변화 이유는 ' + train3['bq20_1'].astype(str) + '이다.'
train3['bq22'] = '이전직업은 ' + train3['bq22'].astype(str) + '이다.'
train3['bq23'] = '전직 가능 직업은 ' + train3['bq23'].astype(str) + '이다.'
train3['bq24'] = '신직업 발굴은 ' + train3['bq24'].astype(str) + '이다.'
train3['bq27_1'] = '나의 전공은 ' + train3['bq27_1'].astype(str) + '이다.'

temp = ''
for col in cols:
  temp += train3[col]
  temp += ' '

train3['text'] = temp

In [58]:
# bq4_1a, bq4_1b, bq4_1c, bq5_2, bq18_10, bq20_1, bq26_1
# bq1, bq2, bq3, bq5_1, bq6, bq7, bq24, bq25, bq26, bq28, bq29, bq30_1, bq30_2, bq30_3

cols = ['bq1', 'bq2', 'bq3', 'bq5_1', 'bq6', 'bq7', 'bq24', 'bq25', 'bq26', 'bq28', 'bq29', 'bq30_1', 'bq30_2', 'bq30_3',
        'bq4_1a', 'bq4_1b', 'bq4_1c', 'bq5_2', 'bq18_10', 'bq20_1', 'bq26_1']

def null_to_no(text : str):
  if text == ' ':
    return '없다'
  else:
    return text

for col in cols:
  train4[col] = train4[col].map(null_to_no)

train4['bq4_1a'] = '요구자격증은 ' + train4['bq4_1a'].astype(str) + '이다.'
train4['bq4_1b'] = '요구자격증은 ' + train4['bq4_1b'].astype(str) + '이다.'
train4['bq4_1c'] = '요구자격증은 ' + train4['bq4_1c'].astype(str) + '이다.'
train4['bq5_2'] = '요구훈련종류는 ' + train4['bq5_2'].astype(str) + '이다.'
train4['bq18_10'] = '향후 일자리 변화 관련 예시는 ' + train4['bq18_10'].astype(str) + '이다.'
train4['bq20_1'] = '향후 일자리 변화 이유는 ' + train4['bq20_1'].astype(str) + '이다.'
train4['bq26_1'] = '나의 전공은 ' + train4['bq26_1'].astype(str) + '이다.'

temp = ''
for col in cols:
  temp += train4[col]
  temp += ' '

train4['text'] = temp

In [59]:
df = pd.concat([train1[['idx', 'text', 'knowcode']], train2[['idx', 'text', 'knowcode']], train3[['idx', 'text', 'knowcode']], train4[['idx', 'text', 'knowcode']]], axis=0)
df = df.reset_index(drop=True)
df

,idx,text,knowcode
0,0,산업유형은 제조업. 근무업체 유형은 민간회사 또는 개인사업체(외국인회사 포함). 근...,825101
1,1,"산업유형은 전문, 과학 및 기술 서비스업. 근무업체 유형은 민간회사 또는 개인사업체...",140204
2,2,"산업유형은 전문, 과학 및 기술 서비스업. 근무업체 유형은 민간회사 또는 개인사업체...",140204
3,3,"산업유형은 전문, 과학 및 기술 서비스업. 근무업체 유형은 정부기관 및 산하기관(공...",140601
4,4,산업유형은 건설업. 근무업체 유형은 민간회사 또는 개인사업체(외국인회사 포함). 근...,140204
...,...,...,...
35228,35244,"산업유형은 전문, 과학 및 기술 서비스업. 근무업체 유형은 정부기관 및 산하기관(공...",121105
35229,35245,산업유형은 금융 및 보험업. 근무업체 유형은 민간회사 또는 개인사업체(외국인회사 포...,26102
35230,35246,산업유형은 금융 및 보험업. 근무업체 유형은 민간회사 또는 개인사업체(외국인회사 포...,31301
35231,35247,산업유형은 제조업. 근무업체 유형은 민간회사 또는 개인사업체(외국인회사 포함). 근...,154105


In [60]:
def remove_ddaom(text:str):
  return str(text).replace('"', '')

df['text'] = df['text'].map(remove_ddaom)

def remove_enter(text:str):
  return text.replace('\n', ' ')

df['text'] = df['text'].map(remove_enter)

In [61]:
## CPU
# device = torch.device("cpu")

## GPU
device = torch.device("cuda:0")

In [62]:
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [63]:
my_dict = {}
for i in range(len(df['knowcode'].unique())):
  my_dict[df['knowcode'].unique()[i]] = i

In [64]:
def target_to_label(text:str):
  return my_dict[text]

df['target_label'] = df['knowcode'].map(target_to_label)

In [65]:
dataset_train, dataset_test = train_test_split(df[['idx', 'text', 'target_label']], test_size=0.2, random_state=42, shuffle=True)
dataset_train.to_csv('dataset_train.txt', index=False, header=True, sep='\t', encoding='utf8')
dataset_test.to_csv('dataset_test.txt', index=False, header=True, sep='\t', encoding='utf8')

In [66]:
dataset_train = nlp.data.TSVDataset('/content/dataset_train.txt', field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset('/content/dataset_test.txt', field_indices=[1,2], num_discard_samples=1)

In [67]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [68]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [69]:
## Setting parameters
max_len = 128
batch_size = 8
warmup_ratio = 0.1
num_epochs = 40
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [70]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [71]:
next(iter(data_train))

(array([   2, 1375, 3969, 7601, 5546, 6228, 5330, 2486, 7798,  517,   54,
        2641, 7063, 7921, 7086, 4033,  517,   46, 1071, 2184, 1289, 2724,
        6873,  517,   54, 1225, 6879, 3574, 7921, 7086, 4106, 5564, 2184,
        2642, 5564,  522, 1023, 5577, 4864,  517,   40,  517,   54, 1225,
        6879, 1179, 5760,  611, 6204,  517,  463,  517,  171, 6204,  517,
          54, 5184, 5563, 7086, 4919, 6881, 5782,  517,   54, 3264, 5678,
        7003, 1182, 6575, 2415, 7191, 6035, 7100,  517,   54, 1375, 3264,
        5477, 2872, 7488, 7088, 4004, 2234, 3152,  517,   54, 4348, 5342,
        7086, 2609, 7276,  525, 2658, 6493, 6897, 2225,  517,   54, 1375,
        3264,  872, 5330,  742,  517,   54, 1225, 6706, 5468, 3962, 5760,
        4222, 2415, 3969, 6035, 6896, 3643, 4092, 7848, 7345,  517,   54,
        3936, 5859, 5495, 5760, 4222, 2609,    3], dtype=int32),
 array(128, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0

In [72]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=4)

In [73]:
len(df['target_label'].unique())

585

In [74]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=585,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [75]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [76]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [77]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [78]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [79]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [80]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
train = []
test = []
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    train.append(train_acc / (batch_id+1))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test.append(test_acc / (batch_id+1))
    # torch.cuda.empty_cache()

  0%|          | 0/3524 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 6.225021839141846 train acc 0.0
epoch 1 batch id 201 loss 6.523451805114746 train acc 0.0
epoch 1 batch id 401 loss 6.510109901428223 train acc 0.0009351620947630922
epoch 1 batch id 601 loss 6.519132614135742 train acc 0.0014559068219633944
epoch 1 batch id 801 loss 6.400516986846924 train acc 0.0014044943820224719
epoch 1 batch id 1001 loss 6.4205498695373535 train acc 0.002122877122877123
epoch 1 batch id 1201 loss 6.309789657592773 train acc 0.0031223980016652788
epoch 1 batch id 1401 loss 6.377552032470703 train acc 0.003836545324768023
epoch 1 batch id 1601 loss 6.438394546508789 train acc 0.004059962523422861
epoch 1 batch id 1801 loss 6.247600078582764 train acc 0.004580788450860633
epoch 1 batch id 2001 loss 6.38190221786499 train acc 0.004747626186906547
epoch 1 batch id 2201 loss 6.238936424255371 train acc 0.005736029077691958
epoch 1 batch id 2401 loss 6.378456115722656 train acc 0.006143273635985006
epoch 1 batch id 2601 loss 6.186485290527344 trai

  0%|          | 0/881 [00:00<?, ?it/s]

epoch 1 test acc 0.030930760499432462


  0%|          | 0/3524 [00:00<?, ?it/s]

...

epoch 8 batch id 1 loss 4.163385391235352 train acc 0.375
epoch 8 batch id 201 loss 4.2760233879089355 train acc 0.09825870646766169
epoch 8 batch id 401 loss 4.341824531555176 train acc 0.09320448877805486
epoch 8 batch id 601 loss 3.8921046257019043 train acc 0.09130615640599002
epoch 8 batch id 801 loss 3.2853915691375732 train acc 0.09378901373283395
epoch 8 batch id 1001 loss 3.6564979553222656 train acc 0.0979020979020979
epoch 8 batch id 1201 loss 5.3585662841796875 train acc 0.10085345545378852
epoch 8 batch id 1401 loss 4.668491363525391 train acc 0.1026945039257673
epoch 8 batch id 1601 loss 4.618865013122559 train acc 0.10587133041848844
epoch 8 batch id 1801 loss 3.3876702785491943 train acc 0.10653803442531927
epoch 8 batch id 2001 loss 3.519747257232666 train acc 0.1083208395802099
epoch 8 batch id 2201 loss 3.7741522789001465 train acc 0.1100068150840527
epoch 8 batch id 2401 loss 3.9286086559295654 train acc 0.1109954185755935
epoch 8 batch id 2601 loss 3.28873443603515

  0%|          | 0/881 [00:00<?, ?it/s]

epoch 8 test acc 0.11875709421112372


  0%|          | 0/3524 [00:00<?, ?it/s]

epoch 9 batch id 1 loss 4.146994590759277 train acc 0.375
epoch 9 batch id 201 loss 4.171172142028809 train acc 0.11567164179104478
epoch 9 batch id 401 loss 4.2944560050964355 train acc 0.11876558603491272
epoch 9 batch id 601 loss 3.832890272140503 train acc 0.11647254575707154


In [ ]:
torch.save(model, '/content/gdrive/MyDrive/know/model3.pt')

In [ ]:
model = torch.load('/content/gdrive/MyDrive/know/model3.pt')

## 2017 텍스트 데이터 전처리

In [ ]:
def bq1_to_text(num:int):
  if num == 1:
    return '산업유형은 농업, 임업, 어업.'
  elif num == 2:
    return '산업유형은 광업.'
  elif num == 3:
    return '산업유형은 제조업.'
  elif num == 4:
    return '산업유형은 전기, 가스, 증기 및 공기조절 공급업.'
  elif num == 5:
    return '산업유형은 수도·하수·폐기물 처리, 원료재생업.'
  elif num == 6:
    return '산업유형은 건설업.'
  elif num == 7:
    return '산업유형은 도매 및 소매업.'
  elif num == 8:
    return '산업유형은 운수 및 창고업.'
  elif num == 9:
    return '산업유형은 숙박 및 음식점업.'
  elif num == 10:
    return '산업유형은 정보통신업.'
  elif num == 11:
    return '산업유형은 금융 및 보험업.'
  elif num == 12:
    return '산업유형은 부동산업.'
  elif num == 13:
    return '산업유형은 전문, 과학 및 기술 서비스업.'
  elif num == 14:
    return '산업유형은 사업시설 관리, 사업지원 및 임대 서비스업.'
  elif num == 15:
    return '산업유형은 공공행정, 국방 및 사회보장 행정.'
  elif num == 16:
    return '산업유형은 교육 서비스업.'
  elif num == 17:
    return '산업유형은 보건업 및 사회복지 서비스업.'
  elif num == 18:
    return '산업유형은 예술, 스포츠 및 여가관련 서비스업.'
  elif num == 19:
    return '산업유형은 협회 및 단체, 수리 및 기타 개인 서비스업.'
  elif num == 20:
    return '산업유형은 가구내 고용활동 및 달리 분류되지 않은 자가소비 생산활동.'
  elif num == 21:
    return '산업유형은 국제 및 외국기관.'

test1['bq1'] = test1['bq1'].map(bq1_to_text)

In [ ]:
def bq2_to_text(num:int):
  if num == 1:
    return '근무업체 유형은 민간회사 또는 개인사업체(외국인회사 포함).'
  elif num == 2:
    return '근무업체 유형은 정부기관 및 산하기관(공기업 포함).'
  elif num == 3:
    return '근무업체 유형은 (재단, 사단)법인단체.'
  elif num == 4:
    return '근무업체 유형은 특정회사나 사업체에 소속되어 있지 않음 (자영업 포함).'
  elif num == 5:
    return '근무업체 유형은 학교법인 혹은 학원.'
  elif num == 6:
    return '근무업체 유형은 기타.'

test1['bq2'] = test1['bq2'].map(bq2_to_text)

In [ ]:
def bq3_to_text(num:int):
  if num == 1:
    return '근무업체 규모는 1명 ~ 4명.'
  elif num == 2:
    return '근무업체 규모는 5명 ~ 49명.'
  elif num == 3:
    return '근무업체 규모는 50명 ~ 99명.'
  elif num == 4:
    return '근무업체 규모는 100명 ~299명.'
  elif num == 5:
    return '근무업체 규모는 300명 ~ 999명.'
  elif num == 6:
    return '근무업체 규모는 1,000명 이상.'

test1['bq3'] = test1['bq3'].map(bq3_to_text)

In [ ]:
def bq5_1_to_text(num:int):
  try :
    num = int(num)
    if num == 1:
      return '훈련기간은 1개월 미만.'
    elif num == 2:
      return '훈련기간은 1개월 이상~3개월 미만.'
    elif num == 3:
      return '훈련기간은 3개월 이상~6개월 미만.'
    elif num == 4:
      return '훈련기간은 6개월 이상~1년 미만.'
    elif num == 5:
      return '훈련기간은 1년 이상~2년 미만.'
    elif num == 6:
      return '훈련기간은 2년 이상.'
  except :
    return '훈련기간은 필요없다.'

test1['bq5_1'] = test1['bq5_1'].map(bq5_1_to_text)

In [ ]:
def bq6_to_text(num:int):
  if num == 1:
    return '관련된 업무경험은 1개월 미만.'
  elif num == 2:
    return '관련된 업무경험은 1개월 이상~3개월 미만.'
  elif num == 3:
    return '관련된 업무경험은 3개월 이상~6개월 미만.'
  elif num == 4:
    return '관련된 업무경험은 6개월 이상~1년 미만.'
  elif num == 5:
    return '관련된 업무경험은 1년 이상~2년 미만.'
  elif num == 6:
    return '관련된 업무경험은 2년 이상.'
  elif num == 7:
    return '이전 업무경험 필요없음.'

test1['bq6'] = test1['bq6'].map(bq6_to_text)

In [ ]:
def bq7_to_text(num:int):
  if num == 1:
    return '업무 수행 교육수준은 중학교 졸업 이하.'
  elif num == 2:
    return '업무 수행 교육수준은 고등학교 졸업.'
  elif num == 3:
    return '업무 수행 교육수준은 고등학교 졸업 후 직업교육(6개월 이상) 이수.'
  elif num == 4:
    return '업무 수행 교육수준은 전문대학교 졸업.'
  elif num == 5:
    return '업무 수행 교육수준은 대학교 졸업.'
  elif num == 6:
    return '업무 수행 교육수준은 대학원 석사 졸업.'
  elif num == 7:
    return '업무 수행 교육수준은 대학원 박사 졸업.'

test1['bq7'] = test1['bq7'].map(bq7_to_text)

In [ ]:
def bq17_to_text(num:int):
  if num == 1:
    return '업무성과 극대화 연령대은 20대 이하.'
  elif num == 2:
    return '업무성과 극대화 연령대은 30대.'
  elif num == 3:
    return '업무성과 극대화 연령대은 40대.'
  elif num == 4:
    return '업무성과 극대화 연령대은 50대.'
  elif num == 5:
    return '업무성과 극대화 연령대은 60대 이상.'

test1['bq17'] = test1['bq17'].map(bq17_to_text)

In [ ]:
def bq23_to_text(num:int):
  return '기술적 변화요인으로 인한 업무 대체 비율은 ' + str(num) + '%.'

test1['bq23'] = test1['bq23'].map(bq23_to_text)

In [ ]:
def bq36_to_text(num:int):
  if num == 1:
    return '나의 성별은 남성이다.'
  elif num == 2:
    return '나의 성별은 여성이다.'

test1['bq36'] = test1['bq36'].map(bq36_to_text)

In [ ]:
def bq37_to_text(num:int):
  return '나의 나이는 ' + str(num) + '살이다.'

test1['bq37'] = test1['bq37'].map(bq37_to_text)

In [ ]:
def bq38_to_text(num:int):
  if num == 1:
    return '나의 최종학력은 중학교 졸업 이하이다.'
  elif num == 2:
    return '나의 최종학력은 고등학교 졸업이다.'
  elif num == 3:
    return '나의 최종학력은 2~3년제 대학교 졸업이다.'
  elif num == 4:
    return '나의 최종학력은 대학교 졸업이다.'
  elif num == 5:
    return '나의 최종학력은 대학원 석사 졸업이다.'
  elif num == 6:
    return '나의 최종학력은 대학원 박사 졸업이다.'

test1['bq38'] = test1['bq38'].map(bq38_to_text)

In [ ]:
def bq40_to_text(num:int):
  try:
    num = int(num)
    if num == 1:
      return '나의 계약형태는 정규직이다.'
    elif num == 2:
      return '나의 계약형태는 비정규직이다.'
  except:
    return '나의 계약형태는 알 수 없다.'

test1['bq40'] = test1['bq40'].map(bq40_to_text)

In [ ]:
def bq41_1_to_text(num:int):
  try:
    num = int(num)
    return '나의 근로소득은 ' + str(num) + '만원이다.'
  except:
    return '나의 근로소득은 알 수 없다.'

test1['bq41_1'] = test1['bq41_1'].map(bq41_1_to_text)

In [ ]:
def bq41_2_to_text(num:int):
  try:
    num = int(num)
    return '나의 초임임금은 ' + str(num) + '만원이다.'
  except:
    return '나의 초임임금은 알 수 없다.'

test1['bq41_2'] = test1['bq41_2'].map(bq41_2_to_text)

In [ ]:
def bq41_3_to_text(num:int):
  try:
    num = int(num)
    return '나의 순수입은 ' + str(num) + '만원이다.'
  except:
    return '나의 순수입은 알 수 없다.'

test1['bq41_3'] = test1['bq41_3'].map(bq41_3_to_text)

In [ ]:
test1.head()

## 2018 텍스트 데이터 전처리

In [ ]:
def cq48_to_text(num:int):
  if num == 1:
    return '나는 재택근무가 불가능하다.'
  elif num == 2:
    return '나는 재택근무가 가능하다.'

test2['cq48'] = test2['cq48'].map(cq48_to_text)

In [ ]:
def bq1_to_text(num:int):
  if num == 1:
    return '산업유형은 농업, 임업, 어업.'
  elif num == 2:
    return '산업유형은 광업.'
  elif num == 3:
    return '산업유형은 제조업.'
  elif num == 4:
    return '산업유형은 전기, 가스, 증기 및 공기조절 공급업.'
  elif num == 5:
    return '산업유형은 수도·하수·폐기물 처리, 원료재생업.'
  elif num == 6:
    return '산업유형은 건설업.'
  elif num == 7:
    return '산업유형은 도매 및 소매업.'
  elif num == 8:
    return '산업유형은 운수 및 창고업.'
  elif num == 9:
    return '산업유형은 숙박 및 음식점업.'
  elif num == 10:
    return '산업유형은 정보통신업.'
  elif num == 11:
    return '산업유형은 금융 및 보험업.'
  elif num == 12:
    return '산업유형은 부동산업.'
  elif num == 13:
    return '산업유형은 전문, 과학 및 기술 서비스업.'
  elif num == 14:
    return '산업유형은 사업시설 관리, 사업지원 및 임대 서비스업.'
  elif num == 15:
    return '산업유형은 공공행정, 국방 및 사회보장 행정.'
  elif num == 16:
    return '산업유형은 교육 서비스업.'
  elif num == 17:
    return '산업유형은 보건업 및 사회복지 서비스업.'
  elif num == 18:
    return '산업유형은 예술, 스포츠 및 여가관련 서비스업.'
  elif num == 19:
    return '산업유형은 협회 및 단체, 수리 및 기타 개인 서비스업.'
  elif num == 20:
    return '산업유형은 가구내 고용활동 및 달리 분류되지 않은 자가소비 생산활동.'
  elif num == 21:
    return '산업유형은 국제 및 외국기관.'

test2['bq1'] = test2['bq1'].map(bq1_to_text)

In [ ]:
def bq2_to_text(num:int):
  if num == 1:
    return '근무업체 유형은 민간회사 또는 개인사업체(외국인회사 포함).'
  elif num == 2:
    return '근무업체 유형은 정부기관 및 산하기관(공기업 포함).'
  elif num == 3:
    return '근무업체 유형은 (재단, 사단)법인단체.'
  elif num == 4:
    return '근무업체 유형은 특정회사나 사업체에 소속되어 있지 않음 (자영업 포함).'
  elif num == 5:
    return '근무업체 유형은 학교법인 혹은 학원.'
  elif num == 6:
    return '근무업체 유형은 기타.'

test2['bq2'] = test2['bq2'].map(bq2_to_text)

In [ ]:
def bq3_to_text(num:int):
  if num == 1:
    return '근무업체 규모는 1명 ~ 4명.'
  elif num == 2:
    return '근무업체 규모는 5명 ~ 49명.'
  elif num == 3:
    return '근무업체 규모는 50명 ~ 99명.'
  elif num == 4:
    return '근무업체 규모는 100명 ~299명.'
  elif num == 5:
    return '근무업체 규모는 300명 ~ 999명.'
  elif num == 6:
    return '근무업체 규모는 1,000명 이상.'

test2['bq3'] = test2['bq3'].map(bq3_to_text)

In [ ]:
def bq5_1_to_text(num:int):
  try :
    num = int(num)
    if num == 1:
      return '훈련기간은 1개월 미만.'
    elif num == 2:
      return '훈련기간은 1개월 이상~3개월 미만.'
    elif num == 3:
      return '훈련기간은 3개월 이상~6개월 미만.'
    elif num == 4:
      return '훈련기간은 6개월 이상~1년 미만.'
    elif num == 5:
      return '훈련기간은 1년 이상~2년 미만.'
    elif num == 6:
      return '훈련기간은 2년 이상.'
  except :
    return '훈련기간은 필요없다.'

test2['bq5_1'] = test2['bq5_1'].map(bq5_1_to_text)

In [ ]:
def bq13_to_text(num:int):
  if num == 1:
    return '업무내용 규정성은 사업주/상사에게 있다.'
  elif num == 2:
    return '업무내용 규정성은 고객에게 있다.'
  elif num == 3:
    return '업무내용 규정성은 본인재량이다.'

test2['bq13'] = test2['bq13'].map(bq13_to_text)

In [ ]:
def bq14_to_text(num:int):
  if num == 1:
    return '나는 업무관련 수칙을 적용 받는다.'
  elif num == 2:
    return '나는 업무관련 수칙을 적용 받지 않는다.'

test2['bq14'] = test2['bq14'].map(bq14_to_text)

In [ ]:
def bq15_to_text(num:int):
  if num == 1:
    return '지휘감독은 사업주/상사에게 받는다.'
  elif num == 2:
    return '지휘감독은 지점/대리점의 점장이나 소장에게 받는다.'
  elif num == 3:
    return '지휘감독은 고객에게 받는다.'
  elif num == 4:
    return '지휘감독을 받지 않는다.'

test2['bq15'] = test2['bq15'].map(bq15_to_text)

In [ ]:
def bq16_to_text(num:int):
  if num == 1:
    return '나는 업무 거부가 가능하다.'
  elif num == 2:
    return '나는 업무 거부가 불가능하다.'

test2['bq16'] = test2['bq16'].map(bq16_to_text)

In [ ]:
def bq17_to_text(num:int):
  if num == 1:
    return '근무시간과 장소는 주로 사업주/상사에 의해 정해진다.'
  elif num == 2:
    return '근무시간과 장소는 주로 고객에 의해 정해진다.'
  elif num == 3:
    return '근무시간과 장소는 주로 본인 재량에 의해 정해진다.'

test2['bq17'] = test2['bq17'].map(bq17_to_text)

In [ ]:
def bq18_to_text(num:int):
  if num == 1:
    return '작업도구는 주로 사업주/상사 소유다.'
  elif num == 2:
    return '작업도구는 사업주와 본인이 반반 소유한다.'
  elif num == 3:
    return '작업도구는 주로 본인이 소유한다.'

test2['bq18'] = test2['bq18'].map(bq18_to_text)

In [ ]:
def bq21_to_text(num:int):
  return '나의 보수 중 고정급은 ' + str(num) + '%이다.'

test2['bq21'] = test2['bq21'].map(bq21_to_text)

In [ ]:
def bq27_to_text(num:int):
  if num == 1:
    return '나는 플랫폼 중개로 일거리를 얻는다.'
  elif num == 2:
    return '나는 플랫폼 중개로 일거리를 얻지 않는다.'

test2['bq27'] = test2['bq27'].map(bq27_to_text)

In [ ]:
test2.head()

## 2019 텍스트 데이터 전처리

In [ ]:
def bq1_to_text(num:int):
  if num == 1:
    return '산업유형은 농업, 임업, 어업.'
  elif num == 2:
    return '산업유형은 광업.'
  elif num == 3:
    return '산업유형은 제조업.'
  elif num == 4:
    return '산업유형은 전기, 가스, 증기 및 공기조절 공급업.'
  elif num == 5:
    return '산업유형은 수도·하수·폐기물 처리, 원료재생업.'
  elif num == 6:
    return '산업유형은 건설업.'
  elif num == 7:
    return '산업유형은 도매 및 소매업.'
  elif num == 8:
    return '산업유형은 운수 및 창고업.'
  elif num == 9:
    return '산업유형은 숙박 및 음식점업.'
  elif num == 10:
    return '산업유형은 정보통신업.'
  elif num == 11:
    return '산업유형은 금융 및 보험업.'
  elif num == 12:
    return '산업유형은 부동산업.'
  elif num == 13:
    return '산업유형은 전문, 과학 및 기술 서비스업.'
  elif num == 14:
    return '산업유형은 사업시설 관리, 사업지원 및 임대 서비스업.'
  elif num == 15:
    return '산업유형은 공공행정, 국방 및 사회보장 행정.'
  elif num == 16:
    return '산업유형은 교육 서비스업.'
  elif num == 17:
    return '산업유형은 보건업 및 사회복지 서비스업.'
  elif num == 18:
    return '산업유형은 예술, 스포츠 및 여가관련 서비스업.'
  elif num == 19:
    return '산업유형은 협회 및 단체, 수리 및 기타 개인 서비스업.'
  elif num == 20:
    return '산업유형은 가구내 고용활동 및 달리 분류되지 않은 자가소비 생산활동.'
  elif num == 21:
    return '산업유형은 국제 및 외국기관.'

test3['bq1'] = test3['bq1'].map(bq1_to_text)

In [ ]:
def bq2_to_text(num:int):
  if num == 1:
    return '근무업체 유형은 민간회사 또는 개인사업체(외국인회사 포함).'
  elif num == 2:
    return '근무업체 유형은 정부기관 및 산하기관(공기업 포함).'
  elif num == 3:
    return '근무업체 유형은 (재단, 사단)법인단체.'
  elif num == 4:
    return '근무업체 유형은 특정회사나 사업체에 소속되어 있지 않음 (자영업 포함).'
  elif num == 5:
    return '근무업체 유형은 학교법인 혹은 학원.'
  elif num == 6:
    return '근무업체 유형은 기타.'

test3['bq2'] = test3['bq2'].map(bq2_to_text)

In [ ]:
def bq3_to_text(num:int):
  if num == 1:
    return '근무업체 규모는 1명 ~ 4명.'
  elif num == 2:
    return '근무업체 규모는 5명 ~ 49명.'
  elif num == 3:
    return '근무업체 규모는 50명 ~ 99명.'
  elif num == 4:
    return '근무업체 규모는 100명 ~299명.'
  elif num == 5:
    return '근무업체 규모는 300명 ~ 999명.'
  elif num == 6:
    return '근무업체 규모는 1,000명 이상.'

test3['bq3'] = test3['bq3'].map(bq3_to_text)


In [ ]:
def bq5_1_to_text(num:int):
  try :
    num = int(num)
    if num == 1:
      return '훈련기간은 1개월 미만.'
    elif num == 2:
      return '훈련기간은 1개월 이상~3개월 미만.'
    elif num == 3:
      return '훈련기간은 3개월 이상~6개월 미만.'
    elif num == 4:
      return '훈련기간은 6개월 이상~1년 미만.'
    elif num == 5:
      return '훈련기간은 1년 이상~2년 미만.'
    elif num == 6:
      return '훈련기간은 2년 이상.'
  except :
    return '훈련기간은 필요없다.'

test3['bq5_1'] = test3['bq5_1'].map(bq5_1_to_text)

In [ ]:
test3.head()

## 2020년 텍스트 데이터 전처리

In [ ]:
test4['bq1'] = test4['bq1'].map(bq1_to_text)

In [ ]:
test4['bq2'] = test4['bq2'].map(bq2_to_text)

In [ ]:
test4['bq3'] = test4['bq3'].map(bq3_to_text)

In [ ]:
test4['bq5_1'] = test4['bq5_1'].map(bq5_1_to_text)

In [ ]:
def bq6_to_text(num:int):
  if num == 1:
    return '관련된 업무 경험은 1개월 미만.'
  elif num == 2:
    return '관련된 업무 경험은 1개월 이상~3개월 미만.'
  elif num == 3:
    return '관련된 업무 경험은 3개월 이상~6개월 미만.'
  elif num == 4:
    return '관련된 업무 경험은 6개월 이상~1년 미만.'
  elif num == 5:
    return '관련된 업무 경험은 1년 이상~2년 미만.'
  elif num == 6:
    return '관련된 업무 경험은 2년 이상.'
  elif num == 7:
    return '관련된 업무 경험은 필요없음.'

test4['bq6'] = test4['bq6'].map(bq6_to_text)

In [ ]:
def bq7_to_text(num:int):
  if num == 1:
    return '업무 수행 교육수준은 중학교 졸업 이하.'
  elif num == 2:
    return '업무 수행 교육수준은 고등학교 졸업.'
  elif num == 3:
    return '업무 수행 교육수준은 고등학교 졸업 후 직업교육(6개월 이상) 이수.'
  elif num == 4:
    return '업무 수행 교육수준은 전문대학교 졸업.'
  elif num == 5:
    return '업무 수행 교육수준은 대학교 졸업.'
  elif num == 6:
    return '업무 수행 교육수준은 대학원 석사 졸업.'
  elif num == 7:
    return '업무 수행 교육수준은 대학원 박사 졸업.'

test4['bq7'] = test4['bq7'].map(bq7_to_text)

In [ ]:
def bq24_to_text(num:int):
  if num == 1:
    return '나의 성별은 남성이다.'
  elif num == 2:
    return '나의 성별은 여성이다.'

test4['bq24'] = test4['bq24'].map(bq24_to_text)

In [ ]:
def bq25_to_text(num:int):
  return '나의 나이는 ' + str(num) + '살이다.'

test4['bq25'] = test4['bq25'].map(bq25_to_text)

In [ ]:
def bq26_to_text(num:int):
  if num == 1:
    return '나의 최종학력은 중학교 졸업 이하이다.'
  elif num == 2:
    return '나의 최종학력은 고등학교 졸업이다.'
  elif num == 3:
    return '나의 최종학력은 2~3년제 대학교 졸업이다.'
  elif num == 4:
    return '나의 최종학력은 대학교 졸업이다.'
  elif num == 5:
    return '나의 최종학력은 대학원 석사 졸업이다.'
  elif num == 6:
    return '나의 최종학력은 대학원 박사 졸업이다.'

test4['bq26'] = test4['bq26'].map(bq26_to_text)

In [ ]:
def bq28_to_text(num:int):
  try:
    num = int(num)
    if num == 1:
      return '나의 계약형태는 정규직이다.'
    elif num == 2:
      return '나의 계약형태는 비정규직이다.'
  except:
    return '나의 계약형태는 알 수 없다.'

test4['bq28'] = test4['bq28'].map(bq28_to_text)

In [ ]:
def bq29_to_text(num:int):
  return '주당 근로시간은 ' + str(num) + '시간이다.'

test4['bq29'] = test4['bq29'].map(bq29_to_text)

In [ ]:
def bq30_1_to_text(num:int):
  try:
    num = int(num)
    return '나의 근로소득은 ' + str(num) + '만원이다.'
  except:
    return '나의 근로소득은 알 수 없다.'

test4['bq30_1'] = test4['bq30_1'].map(bq30_1_to_text)

In [ ]:
def bq30_2_to_text(num:int):
  try:
    num = int(num)
    return '나의 초임임금은 ' + str(num) + '만원이다.'
  except:
    return '나의 초임임금은 알 수 없다.'

test4['bq30_2'] = test4['bq30_2'].map(bq30_2_to_text)

In [ ]:
def bq30_3_to_text(num:int):
  try:
    num = int(num)
    return '나의 순수입은 ' + str(num) + '만원이다.'
  except:
    return '나의 순수입은 알 수 없다.'

test4['bq30_3'] = test4['bq30_3'].map(bq30_3_to_text)

In [ ]:
test4.head()

In [ ]:
#bq4_1a, bq4_1b, bq4_1c, bq5_2, bq19_1, bq30, bq31, bq32, bq33, bq34, bq38_1
#bq1, bq2, bq3, bq5_1, bq6, bq7, bq17, bq23, bq36, bq37, bq38, bq40, bq41_1, bq41_2, bq41_3

cols = ['bq1', 'bq2', 'bq3', 'bq5_1', 'bq6', 'bq7', 'bq17', 'bq23', 'bq36', 'bq37', 'bq38', 'bq40', 'bq41_1', 'bq41_2', 'bq41_3',
        'bq4_1a', 'bq4_1b', 'bq4_1c', 'bq5_2', 'bq19_1', 'bq30', 'bq31', 'bq32', 'bq33', 'bq34', 'bq38_1']

def null_to_no(text : str):
  if text == ' ':
    return '없다'
  else:
    return text

for col in cols:
  test1[col] = test1[col].map(null_to_no)

test1['bq4_1a'] = '요구자격증은 ' + test1['bq4_1a'].astype(str) + '이다.'
test1['bq4_1b'] = '요구자격증은 ' + test1['bq4_1b'].astype(str) + '이다.'
test1['bq4_1c'] = '요구자격증은 ' + test1['bq4_1c'].astype(str) + '이다.'
test1['bq5_2'] = '요구훈련종류는 ' + test1['bq5_2'].astype(str) + '이다.'
test1['bq19_1'] = '향후 일자리 변화 이유는 ' + test1['bq19_1'].astype(str) + '이다.'
test1['bq30'] = '유사직업명은 ' + test1['bq30'].astype(str) + '이다.'
test1['bq31'] = '사용도구 및 프로그램은 ' + test1['bq31'].astype(str) + '이다.'
test1['bq32'] = '이전직업은 ' + test1['bq32'].astype(str) + '이다.'
test1['bq33'] = '전직 가능 직업은 ' + test1['bq33'].astype(str) + '이다.'
test1['bq34'] = '신직업 발굴은 ' + test1['bq34'].astype(str) + '이다.'
test1['bq38_1'] = '나의 전공은 ' + test1['bq38_1'].astype(str) + '이다.'

temp = ''
for col in cols:
  temp += test1[col]
  temp += ' '

test1['text'] = temp

In [ ]:
# bq4_1a, bq4_1b, bq4_1c, bq5_2, bq28_1,	bq29,	bq30,	bq31,	bq32, bq33, bq37_1
# cq48, bq1, bq2, bq3, bq5_1, bq13, bq14, bq15, bq16, bq17, bq18, bq21, bq27

cols = ['cq48', 'bq1', 'bq2', 'bq3', 'bq5_1', 'bq13', 'bq14', 'bq15', 'bq16', 'bq17', 'bq18', 'bq21', 'bq27',
        'bq4_1a', 'bq4_1b', 'bq4_1c', 'bq5_2', 'bq28_1',	'bq29',	'bq30',	'bq31',	'bq32', 'bq33', 'bq37_1']

def null_to_no(text : str):
  if text == ' ':
    return '없다'
  else:
    return text

for col in cols:
  test2[col] = test2[col].map(null_to_no)

test2['bq4_1a'] = '요구자격증은 ' + test2['bq4_1a'].astype(str) + '이다.'
test2['bq4_1b'] = '요구자격증은 ' + test2['bq4_1b'].astype(str) + '이다.'
test2['bq4_1c'] = '요구자격증은 ' + test2['bq4_1c'].astype(str) + '이다.'
test2['bq5_2'] = '요구훈련종류는 ' + test2['bq5_2'].astype(str) + '이다.'
test2['bq28_1'] = '향후 일자리 변화 이유는 ' + test2['bq28_1'].astype(str) + '이다.'
test2['bq29'] = '유사직업명은 ' + test2['bq29'].astype(str) + '이다.'
test2['bq30'] = '사용도구 및 프로그램은 ' + test2['bq30'].astype(str) + '이다.'
test2['bq31'] = '이전직업은 ' + test2['bq31'].astype(str) + '이다.'
test2['bq32'] = '전직 가능 직업은 ' + test2['bq32'].astype(str) + '이다.'
test2['bq33'] = '신직업 발굴은 ' + test2['bq33'].astype(str) + '이다.'
test2['bq37_1'] = '나의 전공은 ' + test2['bq37_1'].astype(str) + '이다.'

temp = ''
for col in cols:
  temp += test2[col]
  temp += ' '

test2['text'] = temp

In [ ]:
# bq4_1a, bq4_1b, bq4_1c, bq5_2, bq18_10,	bq20_1, bq22,	bq23,	bq24, bq27_1
# bq1, bq2, bq3, bq5_1

cols = ['bq1', 'bq2', 'bq3', 'bq5_1',
        'bq4_1a', 'bq4_1b', 'bq4_1c', 'bq5_2', 'bq18_10',	'bq20_1', 'bq22',	'bq23',	'bq24', 'bq27_1']

def null_to_no(text : str):
  if text == ' ':
    return '없다'
  else:
    return text

for col in cols:
  test3[col] = test3[col].map(null_to_no)

test3['bq4_1a'] = '요구자격증은 ' + test3['bq4_1a'].astype(str) + '이다.'
test3['bq4_1b'] = '요구자격증은 ' + test3['bq4_1b'].astype(str) + '이다.'
test3['bq4_1c'] = '요구자격증은 ' + test3['bq4_1c'].astype(str) + '이다.'
test3['bq5_2'] = '요구훈련종류는 ' + test3['bq5_2'].astype(str) + '이다.'
test3['bq18_10'] = '향후 일자리 변화 관련 예시는 ' + test3['bq18_10'].astype(str) + '이다.'
test3['bq20_1'] = '향후 일자리 변화 이유는 ' + test3['bq20_1'].astype(str) + '이다.'
test3['bq22'] = '이전직업은 ' + test3['bq22'].astype(str) + '이다.'
test3['bq23'] = '전직 가능 직업은 ' + test3['bq23'].astype(str) + '이다.'
test3['bq24'] = '신직업 발굴은 ' + test3['bq24'].astype(str) + '이다.'
test3['bq27_1'] = '나의 전공은 ' + test3['bq27_1'].astype(str) + '이다.'

temp = ''
for col in cols:
  temp += test3[col]
  temp += ' '

test3['text'] = temp

In [ ]:
# bq4_1a, bq4_1b, bq4_1c, bq5_2, bq18_10, bq20_1, bq26_1
# bq1, bq2, bq3, bq5_1, bq6, bq7, bq24, bq25, bq26, bq28, bq29, bq30_1, bq30_2, bq30_3

cols = ['bq1', 'bq2', 'bq3', 'bq5_1', 'bq6', 'bq7', 'bq24', 'bq25', 'bq26', 'bq28', 'bq29', 'bq30_1', 'bq30_2', 'bq30_3',
        'bq4_1a', 'bq4_1b', 'bq4_1c', 'bq5_2', 'bq18_10', 'bq20_1', 'bq26_1']

def null_to_no(text : str):
  if text == ' ':
    return '없다'
  else:
    return text

for col in cols:
  test4[col] = test4[col].map(null_to_no)

test4['bq4_1a'] = '요구자격증은 ' + test4['bq4_1a'].astype(str) + '이다.'
test4['bq4_1b'] = '요구자격증은 ' + test4['bq4_1b'].astype(str) + '이다.'
test4['bq4_1c'] = '요구자격증은 ' + test4['bq4_1c'].astype(str) + '이다.'
test4['bq5_2'] = '요구훈련종류는 ' + test4['bq5_2'].astype(str) + '이다.'
test4['bq18_10'] = '향후 일자리 변화 관련 예시는 ' + test4['bq18_10'].astype(str) + '이다.'
test4['bq20_1'] = '향후 일자리 변화 이유는 ' + test4['bq20_1'].astype(str) + '이다.'
test4['bq26_1'] = '나의 전공은 ' + test4['bq26_1'].astype(str) + '이다.'

temp = ''
for col in cols:
  temp += test4[col]
  temp += ' '

test4['text'] = temp

In [ ]:
df = pd.concat([test1[['idx', 'text']], test2[['idx', 'text']], test3[['idx', 'text']], test4[['idx', 'text']]], axis=0)
df = df.reset_index(drop=True)
df

In [ ]:
def remove_ddaom(text:str):
  return str(text).replace('"', '')

df['text'] = df['text'].map(remove_ddaom)

def remove_enter(text:str):
  return text.replace('\n', ' ')

df['text'] = df['text'].map(remove_enter)

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(sentence):
  data = [sentence, '0']
  dataset_another = [data]
  logits = 0
  another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
  test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=2)
  
  model.eval()

  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
      token_ids = token_ids.long().to(device)
      segment_ids = segment_ids.long().to(device)

      valid_length= valid_length
      label = label.long().to(device)

      out = model(token_ids, valid_length, segment_ids)

      for i in out:
        logits = i
        logits = logits.detach().cpu().numpy()
        logits = np.argmax(logits)

  return logits

label_answer = []
step = 0
for i in tqdm(df['text']):
  label_answer.append(predict(i))

In [ ]:
submission = pd.read_csv('/content/gdrive/MyDrive/know/sample_submission.csv') # sample submission 불러오기
submission['knowcode'] = label_answer
submission

In [ ]:
your_dict = {v: k for k, v in my_dict.items()}

In [ ]:
def label_to_knowcode(num:int):
  return your_dict[num]

submission['knowcode'] = submission['knowcode'].map(label_to_knowcode)

In [ ]:
submission.to_csv('/content/gdrive/MyDrive/know/submission.csv', index=False, encoding='utf8')